In [4]:
!pip install gymnasium
!pip install tensorflow tf_agents
!pip install numpy -U

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
    --------------------------------------- 0.2/12.9 MB 7.0 MB/s eta 0:00:02
   -- ------------------------------------- 0.8/12.9 MB 9.9 MB/s eta 0:00:02
   ------ --------------------------------- 2.1/12.9 MB 16.7 MB/s eta 0:00:01
   ----------- ---------------------------- 3.6/12.9 MB 20.9 MB/s eta 0:00:01
   ----------------------- ---------------- 7.5/12.9 MB 34.2 MB/s eta 0:00:01
   ---------------------------------- ----- 11.2/12.9 MB 59.5 MB/s eta 0:00:01
   ---------------------------------------  12.9/12.9 MB 73.1 MB/s eta 0:00:01
   ---------------------------------------- 12.9/12.9 MB 59.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autoawq 0.1.6 requires torch>=2.1.0, but you have torch 2.0.1 which is incompatible.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.1.3 which is incompatible.
gradio 3.50.2 requires numpy~=1.0, but you have numpy 2.1.3 which is incompatible.
matplotlib 3.8.1 requires numpy<2,>=1.21, but you have numpy 2.1.3 which is incompatible.
pandas 2.1.3 requires numpy<2,>=1.22.4; python_version < "3.11", but you have numpy 2.1.3 which is incompatible.
scikit-learn 1.3.2 requires numpy<2.0,>=1.17.3, but you have numpy 2.1.3 which is incompatible.
scipy 1.11.3 requires numpy<1.28.0,>=1.21.6, but you have numpy 2.1.3 which is incompatible.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.1.3 which is incompatibl

In [5]:
import tensorflow as tf
from tf_agents.environments import py_environment, tf_environment, tf_py_environment
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts
from tf_agents.agents.ppo import ppo_agent
from tf_agents.networks import actor_distribution_network, value_network
from tf_agents.utils import common
from tf_agents.policies import random_tf_policy
from tf_agents.trajectories import trajectory
from tf_agents.drivers import dynamic_step_driver

# Replay buffer
from tf_agents.replay_buffers import tf_uniform_replay_buffer

import numpy as np
import pandas as pd




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\crist\.conda\envs\llm\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\crist\.conda\envs\llm\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\crist\.conda\envs\llm\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\crist\.conda\envs\llm\lib\site-packages\traitlets\config\application.py", line 1053, in launch_instance
    app.start()
  File "c:\Users\cris

AttributeError: _ARRAY_API not found

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
class PortfolioEnv(py_environment.PyEnvironment):
    def __init__(self, data):
        super(PortfolioEnv, self).__init__()
        self._data = data  # DataFrame contendo retornos e volatilidade
        self._current_step = 0
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(50,), dtype=np.int32, minimum=0, maximum=1, name='action'
        )
        self._observation_spec = array_spec.ArraySpec(
            shape=(100,), dtype=np.float32, name='observation'
        )  # 50 retornos + 50 volatilidades

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        self._current_step = 0
        return ts.restart(self._get_observation())

    def _step(self, action):
        if self._current_step >= len(self._data) - 1:
            return ts.termination(self._get_observation(), reward=0.0)
        
        # Limitar a seleção a 4 ativos
        if np.sum(action) > 4:
            reward = -1.0  # Penalidade por selecionar mais de 4 ativos
            return ts.termination(self._get_observation(), reward)
        
        # Calcular retorno do portfólio
        selected = self._data.iloc[self._current_step][['retorno', 'volatilidade']].values
        selected = selected[action == 1]
        if len(selected) == 0:
            reward = 0.0
        else:
            # Exemplo simplificado: média dos retornos anualizados
            reward = np.mean(selected[:, 0])
        
        self._current_step += 1
        if self._current_step >= len(self._data):
            return ts.termination(self._get_observation(), reward)
        else:
            return ts.transition(self._get_observation(), reward=reward, discount=1.0)

    def _get_observation(self):
        # Concatenar retornos e volatilidades
        retornos = self._data.iloc[self._current_step]['retorno'].values
        volatilidades = self._data.iloc[self._current_step]['volatilidade'].values
        return np.concatenate([retornos, volatilidades]).astype(np.float32)


In [ ]:
# Supondo que você já tenha um DataFrame 'df' com colunas 'retorno' e 'volatilidade' para os 50 ativos
# Cada linha representa um estado em um passo de tempo

# Exemplo de preparação:
# df = pd.read_csv('dados.csv')
# df deve ter uma estrutura onde cada linha contém os retornos e volatilidades dos 50 ativos

# Exemplo simplificado:
num_assets = 50
num_steps = 1000  # Número de passos de tempo

# Gerar dados fictícios para exemplificação
np.random.seed(42)
data = {
    'retorno': [np.random.uniform(-0.1, 0.2, num_assets) for _ in range(num_steps)],
    'volatilidade': [np.random.uniform(0.1, 0.3, num_assets) for _ in range(num_steps)]
}
df = pd.DataFrame(data)


In [ ]:
env = PortfolioEnv(df)
tf_env = tf_py_environment.TFPyEnvironment(env)

actor_net = actor_distribution_network.ActorDistributionNetwork(
    tf_env.observation_spec(),
    tf_env.action_spec(),
    fc_layer_params=(256, 128)
)

value_net = value_network.ValueNetwork(
    tf_env.observation_spec(),
    fc_layer_params=(256, 128)
)


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

train_step_counter = tf.Variable(0)

agent = ppo_agent.PPOAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    optimizer,
    actor_net=actor_net,
    value_net=value_net,
    num_epochs=10,
    train_step_counter=train_step_counter
)

agent.initialize()


In [ ]:


replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=tf_env.batch_size,
    max_length=10000
)

# Política de coleta de dados
collect_policy = agent.collect_policy

# Função de coleta
def collect_step(environment, policy, buffer):
    time_step = environment.current_time_step()
    action_step = policy.action(time_step)
    next_time_step = environment.step(action_step.action)
    traj = trajectory.from_transition(time_step, action_step, next_time_step)
    buffer.add_batch(traj)

# Coletar dados iniciais
for _ in range(1000):
    collect_step(tf_env, random_tf_policy.RandomTFPolicy(tf_env.time_step_spec(),
                                                          tf_env.action_spec()), replay_buffer)


In [ ]:
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=64,
    num_steps=2
).prefetch(3)
iterator = iter(dataset)


In [ ]:
num_iterations = 10000
log_interval = 200
save_interval = 1000

for _ in range(num_iterations):
    # Coletar dados
    collect_step(tf_env, agent.collect_policy, replay_buffer)
    
    # Obter um batch de dados
    experience, unused_info = next(iterator)
    
    # Treinar o agente
    train_loss = agent.train(experience).loss
    
    # Logging
    if train_step_counter.numpy() % log_interval == 0:
        print(f'Step {train_step_counter.numpy()}: Loss = {train_loss}')
    
    # Salvar o modelo
    if train_step_counter.numpy() % save_interval == 0:
        # Implementar lógica de salvamento, se desejado
        pass


In [ ]:
policy = agent.policy
num_test_steps = 100

state = tf_env.reset()
for _ in range(num_test_steps):
    action = policy.action(state)
    state, reward, done, info = tf_env.step(action.action)
    print(f"Ação tomada: {action.action.numpy()}, Recompensa: {reward.numpy()}")
    if done:
        state = tf_env.reset()
